## Markov Text Generator

Random text generation with a simple Markov language model

### What is a Markov chain?

- A Markov chain is a stochastic process
- Markov chains use an internal state model to judge the probability of any state based on the current state
- Markov chains have the Markov property: The prediction depends only on the current state.

Now, let's build a simple text generator. We want to predict the most probable next word, based on a limited set of current words.

## The implementation

Let's start with a few basics. We'll use `random` to randomize output, and `pathlib.Path` to access text files. The function `sys.getsizeof` is used later to evaluate memory requirements of our language model. Finally, `matplotlib` is used to visualize experimental results, later on. **Note:** You might have to install matplotlib on your system, first, using pip (or similar).

In [43]:
import sys
# !pip install matplotlib
# !pip install jupyter notebook matplotlib
from collections import defaultdict
import random
from pathlib import Path
from sys import getsizeof
%matplotlib notebook
import matplotlib.pyplot as plt

### 1. Preparing the text

First, select an appropriate data set. You'll want a file of plain text. Potentially you'll have to preprocess whatever data you choose to operate on.

#### Task

Find a data set online, prepare it as raw ASCII text, and load it in the code block below.

In [44]:
text_file = Path('./ai_faq_sample.txt')

Next, you'll need a function **_file_to_words_**, which reads the `text_file` above and produces a list of words from the file. This is used as training data, to build the language model.

#### Task

Complete the function below appropriately.

In [45]:
def file_to_words(text_file):
    with open(text_file, 'r') as file:
        text = file.read()
    return text.split()

try:
    words = list(file_to_words(text_file))
except Exception as e:
    print("Something is wrong with your file")
    print(e)
else:
    print("First 100 words:")
    print(words[:100])

First 100 words:
['What', 'is', 'Artificial', 'Intelligence', '(AI)?', 'AI', 'refers', 'to', 'the', 'simulation', 'of', 'human', 'intelligence', 'processes', 'by', 'machines,', 'especially', 'computer', 'systems.', 'These', 'processes', 'include', 'learning,', 'reasoning,', 'and', 'self-correction.', 'What', 'are', 'the', 'types', 'of', 'AI?AI', 'can', 'be', 'categorized', 'into', 'two', 'types:', 'Narrow', 'AI', '(Weak', 'AI)', 'and', 'General', 'AI', '(Strong', 'AI).', 'Narrow', 'AI', 'is', 'designed', 'for', 'a', 'specific', 'task,', 'while', 'General', 'AI', 'aims', 'to', 'perform', 'any', 'intellectual', 'task', 'that', 'a', 'human', 'can', 'do.', 'What', 'are', 'some', 'examples', 'of', 'Narrow', 'AI?Examples', 'of', 'Narrow', 'AI', 'include', 'virtual', 'personal', 'assistants', 'like', 'Siri', 'and', 'Alexa,', 'recommendation', 'systems', 'like', 'those', 'used', 'by', 'Netflix', 'and', 'Amazon,', 'and', 'autonomous', 'vehicles.', 'What']


### 2. Tuples: Groups of words

While we have the text subdivided into words now, we want to experiment with various context windows. For that, we need to group words into tuples. Since we want to experiment with the context window, the next function should be generic, taking the size of the context window as a parameter.

#### Example

The sentence "What a lovely day in February" (6 words) split with a context window of three yields four tuples:

- (What, a, lovely)
- (a, lovely, day)
- (lovely, day, in)
- (day, in, February)

**Note:** The last word is later on going to be the predicted word. So, if we split with a window size of three, here, we will later on have a context window of only two words (the third word being the predicted one).

#### Task

Complete the function below.

In [46]:
def tuples(words, n):
    return [tuple(words[i:i+n]) for i in range(len(words)-n+1)]
    
test_sentence = "What a lovely day in February".split()
data = list(tuples(test_sentence, 3))[0]
assert(data == ('What', 'a', 'lovely'))

In [47]:
list(tuples(test_sentence, 2))

[('What', 'a'),
 ('a', 'lovely'),
 ('lovely', 'day'),
 ('day', 'in'),
 ('in', 'February')]

### 3. A primitive database

We will create a primitive database to create the Markov chain from the tuples we just got. We'll use a Python dictionary for that. A dictionary can be used as an adjacency list, where the key is the current node and the value is a list of nodes adjacent to the current node.

#### Example

The sentence "I like coffee, I like tea" with a context of one word would produce the following dictionary:
```
{
  ('I'): [('like')],
  ('like'): [('coffee,'), ('tea')],
  ('coffee,'): [('I')],
  ('tea'): [],
}
```

**Note:** You will have to capture the probabilities of the next node. There's multiple ways of implementing this. Consider, how you will do it, then proceed with your implementation.

#### Task

Complete the following function.

In [48]:
def generate_database(data):
    database = defaultdict(list)
    for t in data:
        key = t[:-1]
        value = t[-1]
        database[key].append(value)
    return database

test_sentence = "I like coffee, I like tea".split()
data = list(tuples(test_sentence, 2))
database = generate_database(data)
print(database)
assert(('tea') in database[('like',)])

defaultdict(<class 'list'>, {('I',): ['like', 'like'], ('like',): ['coffee,', 'tea'], ('coffee,',): ['I']})


### 4. Markov text generation

Now, let's generate some text. Based on a prompt, we will predict next tokens repeatedly, until either a certain amount of text is produced, or the chain reaches an end (i.e., a token with no follow-up tokens).

#### Task

Complete the following function

In [52]:
def generate_markov_text(prompt: str, database: dict, text_size: int = 100):
    front = prompt.split()
    while text_size > 0:
        t = tuple(front[-1:])  # Use the last word as the key
        if t in database:
            word = random.choice(database[t])
        else:
            break  # Exit if the tuple is not found in the database
        front.append(word)
        text_size -= 1
        yield word
words = file_to_words(text_file)
# Create tuples with a context size of 1 (pairs of words)
data = tuples(words, 2)  # Create tuples of size 2 (current word and next word)
# Generate the database
database = generate_database(data)
# Initial prompt
prompt = "what is AI"
# Generate text using the Markov chain
generated_text = " ".join(word for word in generate_markov_text(prompt, database))
print(generated_text)

(Strong AI). Narrow AI aims to learn from Machine Learning is a human intelligence processes include virtual personal assistants like Siri and treatment recommendations), finance (fraud detection and Amazon, and autonomous vehicles. What are some examples of data, often unstructured or unlabeled. What are the types of AI include virtual personal assistants like Siri and General AI is used by Netflix and use it to learn from large amounts of data, often unstructured or unlabeled. What is used in various fields, including healthcare (diagnosis and Amazon, and autonomous vehicles. What are some examples of AI?AI can be categorized into two


In [53]:
generated_text = " ".join(word for word in generate_markov_text(prompt, database))
print(generated_text)

that deals with algorithms inspired by machines, especially computer programs that can do. What are the simulation of human can do. What are some examples of AI aims to the development of AI?AI can do. What are the simulation of Narrow AI aims to perform any intellectual task that enables machines to perform any intellectual task that a specific task, while General AI refers to the structure and use it to the development of AI?AI can be categorized into two types: Narrow AI that deals with algorithms inspired by Netflix and entertainment (content recommendation).


In [54]:
generated_text = " ".join(word for word in generate_markov_text(prompt, database))
print(generated_text)

that deals with algorithms inspired by the development of the development of computer systems. These processes by the types of the types of Narrow AI?Examples of AI that can be categorized into two types: Narrow AI refers to learn from large amounts of Narrow AI aims to learn for themselves. How does Deep Learning that can be categorized into two types: Narrow AI (Strong AI). Narrow AI?Examples of human can be categorized into two types: Narrow AI refers to learn for a subset of Machine Learning differ from Machine Learning?Deep Learning that a human intelligence processes by Netflix and function


In [55]:
generated_text = " ".join(word for word in generate_markov_text(prompt, database))
print(generated_text)

is designed for themselves. How does Deep Learning is a subset of AI?AI is Machine Learning?Deep Learning is Artificial Intelligence (AI)? AI is used by machines, especially computer programs that can access data without being explicitly programmed. It focuses on the types of computer systems. These processes include virtual personal assistants like Siri and self-correction. What are some examples of AI?AI is used by the brain's neural networks. It involves learning from data and use it to learn for a specific task, while General AI is a human intelligence processes include learning, reasoning, and autonomous vehicles. What is used by


In [56]:
generated_text = " ".join(word for word in generate_markov_text(prompt, database))
print(generated_text)

include virtual personal assistants like those used by machines, especially computer systems. These processes by the types of the types of computer programs that a subset of Machine Learning?Deep Learning that can be categorized into two types: Narrow AI is a subset of AI?AI can do. What are some examples of AI (Weak AI) and algorithmic trading), retail (personalized shopping experiences), and autonomous vehicles. What are some examples of human can access data and function of Narrow AI?Examples of data, often unstructured or unlabeled. What are some real-world applications of Machine Learning is used by machines, especially computer systems. These


### 5. Experimentation

Now, let's investigate further. 

#### Task

Use the `getsizeof` function to compare memory requirements for various context windows. Experiment with context windows between 1+1 and 20+1 (the "+1" being the predicted next word, respectively). Use `matplotlib` to draw a bar chart of memory requirements based on the training text data you are using.

In [41]:
import random
import sys
import matplotlib.pyplot as plt
from collections import defaultdict

def memory_usage_by_context_window(text_file, max_window_size=20):
    words = file_to_words(text_file)
    memory_requirements = []

    for context_size in range(1, max_window_size + 1):
        data = tuples(words, context_size + 1)  # context_size + 1 to include the next predicted word
        database = generate_database(data)
        memory_requirements.append(sys.getsizeof(database))

    return memory_requirements

# Generate memory usage data
text_file = Path('./ai_faq_sample.txt')  # Replace with your text file
memory_requirements = memory_usage_by_context_window(text_file)

# Plotting the results
plt.figure(figsize=(10, 6))
plt.bar(range(1, 21), memory_requirements, color='blue')
plt.xlabel('Context Window Size (n+1)')
plt.ylabel('Memory Usage (bytes)')
plt.title('Memory Requirements for Various Context Window Sizes')
plt.xticks(range(1, 21))
# Save the plot to a file instead of displaying it
plt.savefig('db_building_memory_usage_plot.png')
plt.show()


<IPython.core.display.Javascript object>

<p align="center">
    <img src="./text_gen_memory_usage_plot.png" alt="Screenshot memory usage" width="600">
</p>


In [42]:
def generate_markov_text(prompt: str, database: dict, text_size: int = 50):
    front = prompt.split()
    generated_words = list(front)
    while text_size > 0:
        t = tuple(front[-1:])  # Use the last word as the key
        if t in database:
            word = random.choice(database[t])
        else:
            break  # Exit if the tuple is not found in the database
        front.append(word)
        generated_words.append(word)
        text_size -= 1
    return " ".join(generated_words)

def memory_usage_by_context_window(words, max_window_size=20):
    memory_requirements = []
    sample_texts = []

    for context_size in range(1, max_window_size + 1):
        data = tuples(words, context_size + 1)  # context_size + 1 to include the next predicted word
        database = generate_database(data)
        memory_requirements.append(sys.getsizeof(database))

        # Generate sample text for each context window size
        prompt = " ".join(words[:context_size])
        sample_text = generate_markov_text(prompt, database, text_size=50)
        sample_texts.append((context_size, sample_text))

    return memory_requirements, sample_texts

# Load text data
text_file = Path('./ai_faq_sample.txt')  # Replace with your text file
words = file_to_words(text_file)

# Generate memory usage data and sample texts
memory_requirements, sample_texts = memory_usage_by_context_window(words, max_window_size=20)

# Plotting the memory requirements
plt.figure(figsize=(10, 6))
plt.bar(range(1, 21), memory_requirements, color='blue')
plt.xlabel('Context Window Size (n+1)')
plt.ylabel('Memory Usage (bytes)')
plt.title('Memory Requirements for Various Context Window Sizes')
plt.xticks(range(1, 21))
# Save the plot to a file instead of displaying it
plt.savefig('text_gen_memory_usage_plot.png')
plt.show()

# Display sample texts
for context_size, sample_text in sample_texts:
    print(f"Context Size {context_size + 1}:\n{sample_text}\n")

<IPython.core.display.Javascript object>

Context Size 2:
What are the types of computer systems. These processes include virtual personal assistants like those used by machines, especially computer systems. These processes by machines, especially computer systems. These processes by Netflix and self-correction. What are some real-world applications of AI?AI can do. What is designed for themselves. How does Deep

Context Size 3:
What is

Context Size 4:
What is Artificial

Context Size 5:
What is Artificial Intelligence

Context Size 6:
What is Artificial Intelligence (AI)?

Context Size 7:
What is Artificial Intelligence (AI)? AI

Context Size 8:
What is Artificial Intelligence (AI)? AI refers

Context Size 9:
What is Artificial Intelligence (AI)? AI refers to

Context Size 10:
What is Artificial Intelligence (AI)? AI refers to the

Context Size 11:
What is Artificial Intelligence (AI)? AI refers to the simulation

Context Size 12:
What is Artificial Intelligence (AI)? AI refers to the simulation of

Context Size 13:
What is 